In [68]:
import sys, os
from os.path import join as pjoin

import numpy as np
import scipy.stats as stats
import scipy.sparse as sparse
#import scipy.linalg as linalg
#import scipy.sparse.linalg as splinalg
import h5py
import matplotlib.pyplot as plt
import seaborn as sns

In [76]:
# LOAD DATA
root = '/media/hdd1/g4sim/beamlet_dose/HN011/1.5T'
def load_sparse_mat(fp):
    with h5py.File(fp, 'r') as fd:
        d = fd['data'][()]
        i = fd['i'][()]
        j = fd['j'][()]
        ncols = fd['ncols'][()]
        nrows = fd['nrows'][()]
    return sparse.coo_matrix((d, (i, j)), shape=(nrows, ncols)).tocsr()

sp_noisy = load_sparse_mat(pjoin(root, '5000', 'beamlet_dose_noisy.h5'))
sp_pred  = load_sparse_mat(pjoin(root, '5000', 'beamlet_dose_predicted.h5'))
sp_true  = load_sparse_mat(pjoin(root,  '1e5', 'beamlet_dose_true.h5'))


In [78]:
# PRINT STATS
def print_matrix_stats(spmat: sparse.spmatrix):
    assert spmat.nnz == spmat.count_nonzero()
    print('storage:     {:0.1f} MB'.format((spmat.data.nbytes + spmat.indices.nbytes + spmat.indptr.nbytes)/1024.0/1024.0))
    print('shape:       {!s}'.format(spmat.shape))
    print('nnz:         {}'.format(spmat.nnz))
    print('sparsity:    {:0.2%}'.format(float(spmat.nnz) / np.product(spmat.shape)))
    for lb, v in (
        ('mean', spmat.mean()),
        ('min', spmat.min()),
        ('max', spmat.max()),
        ('mean (nz)', np.mean(spmat.data)),
        ('median (nz)', np.median(spmat.data)),
        ('min  (nz)', np.amin(spmat.data)),
        ('max  (nz)', np.amax(spmat.data)),
    ):
        print('{:12s} {:0.3e}'.format(lb.rstrip(':')+':', v))
        
    # fig = plt.figure()
    # ax = fig.add_axes([0,0,1,1])
    # sns.distplot(spmat.data, kde=False, rug=True, ax=ax)
    # plt.show()

print('True M:')
print_matrix_stats(sp_true)
print()
print('Pred M:')
print_matrix_stats(sp_pred)
print()
print('Noisy M:')
print_matrix_stats(sp_noisy)

True M:
storage:     1260.4 MB
shape:       (2408256, 2111)
nnz:         164004500
sparsity:    3.23%
mean:        3.725e-01
min:         0.000e+00
max:         3.107e+03
mean (nz):   1.155e+01
median (nz): 1.882e+00
min  (nz):   7.341e-02
max  (nz):   3.107e+03

Pred M:
storage:     2323.7 MB
shape:       (2408256, 2111)
nnz:         303365548
sparsity:    5.97%
mean:        2.480e-01
min:         0.000e+00
max:         2.368e+03
mean (nz):   4.156e+00
median (nz): 5.156e-01
min  (nz):   6.238e-02
max  (nz):   2.368e+03

Noisy M:
storage:     136.7 MB
shape:       (2408256, 2111)
nnz:         16707856
sparsity:    0.33%
mean:        3.749e-01
min:         0.000e+00
max:         1.108e+04
mean (nz):   1.141e+02
median (nz): 3.125e+01
min  (nz):   2.399e-01
max  (nz):   1.108e+04
